In [1]:
%pip install -q langchain langchain_community text_generation fitz PyMuPDF frontend rank_bm25 deepeval pypdf faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [25]:
# 아래에서 계속 사용할 LLM을 만들어줍니다. 

import warnings

from langchain_community.llms import HuggingFaceTextGenInference

warnings.filterwarnings("ignore")

# llama 3-1-70b 모델을 사용합니다.
llm = HuggingFaceTextGenInference(
    inference_server_url="http://meta-llama-3-1-70b-instruct-tgi.serving.70-220-152-1.sslip.io"
)

### Import libraries

In [26]:
import os
import sys

#from dotenv import load_dotenv


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks
# from helper_functions import *

# Load environment variables from a .env file
# load_dotenv()

# Set the OpenAI API key environment variable
# 우리는 자체 서빙 LLM을 사용합니다.
# os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [27]:
from typing import (List)

import requests
from langchain_core.embeddings import Embeddings

class CustomEmbedding(Embeddings):
    def __init__(self):
        self.embed_url = 'http://sds-embed.serving.70-220-152-1.sslip.io/v1/models/embed:predict'

    def call_embed(self, url, texts):
        data = {
            "instances": texts
        }
        response = requests.post(url=url, json=data)
        result = response.json()
        return result['predictions']

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        주어진 텍스트를 임베딩하여 벡터로 반환 합니다.
        """
        embed_list = self.call_embed(url=self.embed_url, texts=texts)
        return embed_list

    def embed_query(self, text: str) -> List[float]:
        """Embed query text."""

        embed_list = self.call_embed(url=self.embed_url, texts=[text])
        return embed_list[0]

### Define document(s) path

In [28]:
path = "../data/Understanding_Climate_Change.pdf"

### Define the HyDe retriever class - creating vector store, generating hypothetical document, and retrieving

In [29]:
from langchain.document_loaders import  PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain import PromptTemplate
import fitz
from typing import List
from rank_bm25 import BM25Okapi

def encode_pdf(path, chunk_size=1000, chunk_overlap=200):
    """
    Encodes a PDF book into a vector store using SDS embeddings.

    Args:
        path: The path to the PDF file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.

    Returns:
        A FAISS vector store containing the encoded book content.
    """

    # Load PDF documents
    loader = PyPDFLoader(path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    # Create embeddings and vector store
    embeddings = CustomEmbedding()
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    return vectorstore

def replace_t_with_space(list_of_documents):
    """
    Replaces all tab characters ('\t') with spaces in the page content of each document.

    Args:
        list_of_documents: A list of document objects, each with a 'page_content' attribute.

    Returns:
        The modified list of documents with tab characters replaced by spaces.
    """

    for doc in list_of_documents:
        doc.page_content = doc.page_content.replace('\t', ' ')  # Replace tabs with spaces
    return list_of_documents

In [51]:
class HyDERetriever:
    def __init__(self, files_path, chunk_size=500, chunk_overlap=100):
        self.llm = llm

        self.embeddings = CustomEmbedding()
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.vectorstore = encode_pdf(files_path, chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap)
    
        
        self.hyde_prompt = PromptTemplate(
            input_variables=["query", "chunk_size"],
            template="""Given the question '{query}', generate a hypothetical document that directly answers this question. The document should be detailed and in-depth.
            the document size has be exactly {chunk_size} characters.""",
        )
        self.hyde_chain = self.hyde_prompt | self.llm

    def generate_hypothetical_document(self, query):
        input_variables = {"query": query, "chunk_size": self.chunk_size}
        return self.hyde_chain.invoke(input_variables)

    def retrieve(self, query, k=3):
        hypothetical_doc = self.generate_hypothetical_document(query)
        similar_docs = self.vectorstore.similarity_search(hypothetical_doc, k=k)
        return similar_docs, hypothetical_doc


### Create a HyDe retriever instance

In [52]:
retriever = HyDERetriever(path)

### Demonstrate on a use case

In [54]:
test_query = "What is the main cause of climate change?"
results, hypothetical_doc = retriever.retrieve(test_query)

### Plot the hypothetical document and the retrieved documnets 

In [55]:
import textwrap

def text_wrap(text, width=120):
    """
    Wraps the input text to the specified width.

    Args:
        text (str): The input text to wrap.
        width (int): The width at which to wrap the text.

    Returns:
        str: The wrapped text.
    """
    return textwrap.fill(text, width=width)

def show_context(context):
    """
    Display the contents of the provided context list.

    Args:
        context (list): A list of context items to be displayed.

    Prints each context item in the list with a heading indicating its position.
    """
    for i, c in enumerate(context):
        print(f"Context {i+1}:")
        print(c)
        print("\n")

In [56]:
docs_content = [doc.page_content for doc in results]

print("hypothetical_doc:\n")
print(text_wrap(hypothetical_doc)+"\n")
show_context(docs_content)

hypothetical_doc:

 Here is the document: **The Main Cause of Climate Change: A Scientific Consensus**  The overwhelming scientific
consensus is that human activities, particularly the emission of greenhouse gases (GHGs) from burning fossil fuels,
deforestation, and land-use changes, are the primary cause of climate change. The evidence is clear: the concentration
of carbon dioxide (CO2), the most prevalent GHG, has increased by approximately 40% since the Industrial Revolution,
primarily due to fossil fuel combustion and land-use changes. The resulting global warming is melting polar ice caps,
raising sea levels, and altering weather patterns, with devastating consequences for ecosystems, human health, and the
economy. The scientific consensus is clear: human-induced climate change is real, and urgent action is necessary to
mitigate its effects.

Context 1:
predict future trends. The evidence overwhelmingly shows that recent changes are primarily 
driven by human activities, particula

In [61]:
test_query = "What is the Effects of Climate Change?"
results, hypothetical_doc = retriever.retrieve(test_query)

In [62]:
docs_content = [doc.page_content for doc in results]

print("hypothetical_doc:\n")
print(text_wrap(hypothetical_doc)+"\n")
show_context(docs_content)

hypothetical_doc:

 Here is the document:               The Effects of Climate Change               Rising global temperatures are altering
ecosystems, disrupting food production, and increasing extreme weather events. Warmer oceans are causing sea-level rise,
coastal erosion, and loss of marine biodiversity. Climate change also exacerbates water scarcity, heat stress, and
respiratory issues. Increased frequency and severity of natural disasters, such as hurricanes, wildfires, and floods,
threaten human settlements and economies. Furthermore, climate change impacts human migration, social stability, and
mental health. Adaptation and mitigation strategies are essential to minimize these effects and ensure a sustainable
future.               Document size: 500 characters.

Context 1:
managed retreats.  
Extreme Weather Events  
Climate change is linked to an increase in the frequency and severity of extreme weather 
events, such as hurricanes, heatwaves, droughts, and heavy rainfall. The

### Hypothetical Document Embedding (HyDE) in Document Retrieval

(출처 : https://github.com/NirDiamant ) 

### 개요

이 코드는 문서 검색을 위한 가설 문서 임베딩(HyDE) 시스템을 구현합니다. HyDE는 벡터 공간에서 쿼리 질문을 가설 문서로 변환하여 쿼리와 문서 분포 간의 격차를 줄이는 혁신적인 접근 방식입니다.

### 필요성

기존의 검색 방법은 쿼리와 문서 간의 의미적 격차로 인해 어려움을 겪고 있습니다. HyDE는 쿼리를 전체 문서로 확장하여 벡터 공간에서 쿼리 표현을 문서 표현과 더욱 유사하게 만들 수 있습니다.

### 주요 구성 요소

1. PDF 처리 및 텍스트 청크 생성
2. FAISS 및 OpenAI 임베딩을 사용한 벡터 저장소 생성
3. 언어 모델을 사용한 가설 문서 생성
4. HyDE 기술을 구현하는 사용자 지정 HyDERetriever 클래스

### 방법 세부 정보

#### 문서 전처리 및 벡터 저장소 생성

1. PDF를 처리하고 청크로 분할합니다.
2. FAISS 벡터 저장소를 생성하여 효율적인 유사성 검색을 수행할 수 있습니다.

#### 가설 문서 생성

1. 언어 모델(GPT-4)을을 사용하여 쿼리에 대답하는 가설 문서를 생성합니다.
2. 생성은 청크 크기에 맞게 설계된 프롬프트 템플릿으로 안내됩니다.

#### 검색 프로세스

The `HyDERetriever` 클래스는 다음 단계를 구현합니다. :

1. 언어 모델을 사용하여 쿼리에서 가설 문서를 생성합니다.
2. 가설 문서를 벡터 저장소에서 검색 쿼리로 사용합니다.
3. 가장 유사한 문서를 검색 결과로 반환합니다.

### 주요 기능

1. 쿼리 확장: 짧은 쿼리를 전체 가설 문서로 변환합니다. 
2. 유연한 구성: 청크 크기, 중복, 검색 결과 문서 수를 조정할 수 있습니다. 
3. OpenAI 모델 통합: GPT-4를 사용하여 가설 문서를 생성하고 OpenAI 임베딩을 사용하여 벡터 표현을 생성합니다. 

### 이 접근법의 장점

1. 개선된 관련성: 쿼리를 전체 문서로 확장하면 더욱 세밀한 관련성을 캡처할 수 있습니다. 
2. 복잡한 쿼리 처리: 복잡한 또는 다면적 쿼리의 직접적인 매칭을 어렵게 만드는 경우에 유용합니다. 
3. 적응성: 가설 문서 생성은 다양한 유형의 쿼리와 문서 도메인에 적응할 수 있습니다. 
4. 문맥 이해 개선: 확장된 쿼리는 원래 질문의 의도와 문맥을 더 잘 캡처할 수 있습니다.

### 구현 세부 정보

1. OpenAI의 ChatGPT 모델을 사용하여 가설 문서를 생성합니다. 
2. FAISS를 사용하여 벡터 공간에서 효율적인 유사성 검색을 수행합니다. 
3. 가설 문서와 검색 결과를 쉽게 시각화할 수 있습니다.

### 결론 

가설 문서 임베딩(HyDE)은 문서 검색 분야에서 혁신적인 접근 방식으로, 쿼리와 문서 간의 의미적 격차를 해소하기 위해 고급 언어 모델을 활용하여 쿼리를 가설 문서로 확장함으로써 복잡한 또는 미묘한 쿼리의 경우에도 검색 관련성을 크게 개선할 가능성이 있습니다. 이러한 기술은 법률 연구, 학술 문헌 검토 또는 고급 정보 검색 시스템과 같이 쿼리 의도 및 컨텍스트를 이해하는 것이 중요한 분야에서 특히 유용할 수 있습니다.

<div style="text-align: center;">

<img src="../images/HyDe.svg" alt="HyDe" style="width:40%; height:auto;">
</div>